## 12. Modellparameter und SVD
Wir analysieren die Gewichtsmatrix der letzten Schicht mit SVD und zeigen die Form und Werte.

In [ ]:
PlotParameters(model)

## 11. Filter nach Training visualisieren
Wir plotten die Filtergewichte nach dem Training erneut.

In [ ]:
train_accuracyValue = train_accuracy_list[-1]
print(f'{train_accuracyValue=}')
validation_accuracyValue = validation_accuracy_list[-1]
print(f'{validation_accuracyValue=}')
fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.plot(loss_list,color=color)
ax1.set_xlabel('epoch')
ax1.set_ylabel('total loss',color=color)
ax1.tick_params(axis='y', color=color)
ax2 = ax1.twinx()  
color = 'tab:blue'
ax2.set_ylabel('accuracy', color=color)  
ax2.plot( train_accuracy_list, color=color)
ax2.tick_params(axis='y', color=color)
fig.tight_layout()
color = 'tab:orange'
ax2.plot( validation_accuracy_list, color=color)
color = 'tab:purple'
ax2.plot( validation_loss_list, color=color)

## 10. Trainings- und Validierungsmetriken visualisieren
Wir plotten die Loss- und Accuracy-Werte für Training und Validierung.

In [ ]:
for t in range(n_epochs):
    print(f"n_epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_func, optimizer)
    validate(validation_loader, model, loss_func)
print("Done!")

## 9. Training und Validierung durchführen
Wir trainieren das Modell und validieren es nach jeder Epoche.

In [ ]:
loss_list = []
train_accuracy_list = []
N_train = len(train_dataset)
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    optimizer.zero_grad()
    train_loss, correct = 0, 0
    for batchNr, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        _, yhat = torch.max(pred.data, 1)
        correct += (yhat == y).sum().item()
        train_loss += loss
        if batchNr % 100 == 0:
            loss, current = loss.item(), (batchNr + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    accuracy = correct / N_train
    train_accuracy_list.append(accuracy)
    loss_list.append(train_loss.item()/size)

validation_loss_list = []
validation_accuracy_list = []
N_valid = len(validation_dataset)
def validate(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    validation_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            validation_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    validation_loss /= num_batches
    accuracy = correct / N_valid
    validation_accuracy_list.append(accuracy)
    validation_loss_list.append(validation_loss)
    print(f"Test Error: \n Accuracy: {(100*accuracy):>0.1f}%, Avg loss: {validation_loss:>8f} \n")

## 8. Trainings- und Validierungsfunktionen
Wir definieren die Trainings- und Validierungsfunktionen für das Modell.

In [ ]:
n_epochs = 5
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_func = nn.CrossEntropyLoss()
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True, num_workers=1)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, batch_size=500, shuffle=True, num_workers=1)

## 7. Trainingsparameter und DataLoader
Wir definieren die Trainingsparameter, den Optimierer, die Loss-Funktion und die DataLoader für Training und Validierung.

In [ ]:
def PlotParameters(model): 
    W = model.state_dict()['conv1.0.weight'].data.cpu()
    w_min = W.min().item()
    w_max = W.max().item()
    fig, axes = plt.subplots(3, 6)
    fig.subplots_adjust(hspace=0.01, wspace=0.1)
    for i, ax in enumerate(axes.flat):
        if i < 16:
            ax.set_xlabel(f"Filter: {i}")
            ax.imshow(W[i, :].view(5, 5), vmin=w_min, vmax=w_max, cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        if i in range(16,18):
            ax.axis('off')
    plt.show()
PlotParameters(model)

## 6. Filtergewichte visualisieren
Wir plotten die Filtergewichte der ersten Convolution-Schicht.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
if device != "cuda":
    device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f'{device=}')
model = CNN().to(device)
print("Print the model:\n ", model)

## 5. Gerät wählen (CPU/GPU) und Modell erstellen
Wir wählen das Gerät (CPU/GPU/MPS) und initialisieren das Modell.

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.out = nn.Linear(32*7*7, 10)
    def forward(self, input):
        x = self.conv1(input)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        pred = self.out(x)
        return pred

## 4. CNN-Modell definieren
Wir definieren die Architektur des Convolutional Neural Networks (CNN) mit zwei Convolution-Schichten und einer voll verbundenen Schicht.

In [ ]:
def show_data(data_sample):
    plt.imshow(data_sample[0].numpy().squeeze(), cmap='gray')
    plt.title('y = ' + str(data_sample[1]))

## 3. Beispielbild anzeigen
Wir definieren eine Funktion, um ein Beispielbild aus dem Datensatz darzustellen.

In [ ]:
train_dataset = dsets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
print("Print the training dataset:\n ", train_dataset)
validation_dataset = dsets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
print("Print the validating dataset:\n ", validation_dataset)

## 2. MNIST Datensätze laden
Wir laden den Trainings- und Validierungsdatensatz und zeigen deren Eigenschaften.

In [ ]:
import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import matplotlib.pylab as plt
import numpy as np

## 1. Bibliotheken importieren
Wir importieren die benötigten Bibliotheken für das Training und die Visualisierung.

# Convolutional Neural Network (CNN) mit PyTorch für MNIST
Dieses Notebook zeigt, wie man ein einfaches CNN mit PyTorch für die Klassifikation des MNIST-Datensatzes erstellt, trainiert und auswertet.